# Submit Jobrun

## 1. Input Data

In [4]:
import requests
import json
from io import BytesIO
import pandas as pd
from zipfile import ZipFile

job_data = pd.read_csv(r"C:\_Working\PAT_20201019\debug\test_jobrun_input.csv") #data/test_jobrun.csv')
job_data

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup
0,902071_720820,902071,76030019.71,0.25,391213.71,1625918.698,720820,3.912137e+05,1
1,902071_720922,902071,76030019.71,0.25,386318.01,1625918.698,720922,3.863180e+05,1
2,902071_717188,902071,76030019.71,0.25,470508.19,1625918.698,717188,4.705082e+05,1
3,900324_701536,900324,76030019.71,0.25,537895.47,1625918.698,701536,5.378955e+05,1
4,902071_717311,902071,76030019.71,0.25,305977.33,1625918.698,717311,3.059773e+05,1
...,...,...,...,...,...,...,...,...,...
20991,900324_701182,900324,76030019.71,0.25,1318833.46,1625918.698,701182,1.318833e+06,11
20992,900324_701200,900324,76030019.71,0.25,729497.72,1625918.698,701200,7.294977e+05,11
20993,902071_716942,902071,76030019.71,0.25,2991696.49,1625918.698,716942,2.991696e+06,11
20994,902071_716970,902071,76030019.71,0.25,95359.53,1625918.698,716970,9.535953e+04,11


## 2. Input Parameters

In [5]:
job_para = json.load(open('data/test_jobrun.json'))
print(json.dumps(job_para,indent=2))

{
  "valid_rules": 1073741826,
  "loss_alae_ratio": 1,
  "additional_coverage": 2,
  "deductible_treatment": "Retains_Limit",
  "coverage_type": "Building_Contents_BI",
  "type_of_rating": "PSOLD",
  "curve_id": 2016,
  "average_accident_date": "1/1/2022",
  "psold": {
    "curve_persp": "Gross",
    "peril_subline": "All_Perils",
    "trend_factor": 1.035,
    "blending": [
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "hpr_blending": false
  },
  "user_name": "Chengyou Xiao",
  "user_email": "chengyou.xiao@guycarp.com"
}


## 3. Request to the service

In [6]:
files = {
    'para': json.dumps(job_para),
    'data': job_data.to_csv(index=False)
}
resp = requests.post("http://localhost:5000/api/jobrun", files=files)
if resp.status_code == 200:
    with ZipFile(BytesIO(resp.content),'r') as zf:
        df_res =  pd.read_csv(zf.open('pat_res.csv'))

df_res.sort_values('PseudoPolicyID')

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup,LossRatio,Policy,EffLmt,AOIr,PolLAS,DedLAS,Premium,sumLAS
20983,883100_2071035,883100,6.202362e+07,0.225,8.269816e+06,82698.16225,2071035,1.529916e+07,30,1.0,7.029344e+07,4.589748e+07,39,258995.924588,203071.370507,6139.047482,753351.046782
20986,883100_2071036,883100,6.202362e+07,0.225,5.513211e+06,82698.16225,2071036,5.513211e+06,30,1.0,6.753683e+07,1.653963e+07,35,126304.020249,106721.607055,2149.634743,753351.046782
20982,883100_2071038,883100,6.202362e+07,0.225,8.269816e+06,82698.16225,2071038,7.270109e+07,30,1.0,7.029344e+07,2.108803e+08,46,319064.935779,236444.889152,9069.511551,753351.046782
20985,883100_2071039,883100,6.202362e+07,0.225,8.269816e+06,82698.16225,2071039,9.444036e+06,30,1.0,7.029344e+07,2.833211e+07,38,208326.391701,171232.506477,4071.934528,753351.046782
20984,883100_2071040,883100,6.202362e+07,0.225,8.269816e+06,82698.16225,2071040,1.882963e+08,30,1.0,7.029344e+07,2.108803e+08,51,395661.438220,282385.087534,12434.768715,753351.046782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20902,910084_2441648,910084,3.519020e+08,1.000,1.395100e+08,0.00000,2441648,1.395100e+08,32,1.0,4.914120e+08,4.185300e+08,49,423153.345983,405312.507960,0.000000,97810.083968
20904,910084_2441649,910084,3.519020e+08,1.000,2.076281e+08,0.00000,2441649,2.076281e+08,32,1.0,5.595302e+08,6.228844e+08,51,428044.687778,417403.498133,0.000000,97810.083968
20896,910084_2441650,910084,3.519020e+08,1.000,2.363609e+08,0.00000,2441650,3.053678e+08,32,1.0,5.882629e+08,9.161034e+08,52,431099.114102,422693.073266,0.000000,97810.083968
20895,910084_2441651,910084,3.519020e+08,1.000,2.363609e+08,0.00000,2441651,3.098046e+08,32,1.0,5.882629e+08,9.294138e+08,53,435445.626365,427039.415123,0.000000,97810.083968
